## Coarse Graining Polymers using GRiTS

In [3]:
import grits
import mbuild as mb
import inspect
from polymer_dictionary import bond_indices
from polymer_dictionary import polymer_fragments
from polymer_dictionary import element_dict
import os
import glob
import string
from grits import CG_System

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/foyer/forcefield.py:34: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import iter_entry_points, resource_filename
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/pkg_resources/__init__.py:3146: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)



Support for writing out LAMMPS data files will be removed
in mbuild 1.0.
See GMSO (https://github.com/mosdef-hub/gmso/tree/main/gmso/formats/lammpsdata) for
continued support for LAMMPS.



In [4]:
key_list = sorted(list(bond_indices.keys())) # Aligning dictionary and path to aid automation
path = (os.getcwd()+"/mol2/") # Non-user specific path
molecule_list = sorted(glob.glob(path+"*.mol2"))
molecule_list.pop(10) # Removing polymer not tested
print(key_list)

['PCPDTFBT_C11_BO', 'PCPDTFBT_C1_BO', 'PCPDTFBT_C3_BO', 'PCPDTFBT_C4_BO', 'PCPDTFBT_C5_BO', 'PCPDTPT_HD', 'PCPDTPT_ODD', 'PCPDTPT_eneODD', 'PCPDTPT_nC16', 'PCPDT_PT_eneHD', 'PIDTBT_nC16', 'PIDTCPDT_C11BO', 'PIDTFBT_C11_BO']


In [7]:
# Function to parse through GSD nomenclature and automatically get backbone segments
def cg(filename):
    letters = string.ascii_uppercase
    name = os.path.basename(filename)
    name = (os.path.splitext(name)[0])[1:]
    new_gsd_name = "" + name
    
    final_cg = [polymer_fragments["P"]]
    grits_string = {}
    if name[0:4] == "CPDT":
        final_cg.append(polymer_fragments["CPDT"])
        name = name[4:]
    if name[0:3] == "IDT":
        final_cg.append(polymer_fragments["IDT"])
        name = name[3:]
    if name[0:3] == "FBT":
        final_cg.append(polymer_fragments["FBT"])
        name = name[3:]
    if name[0:2] == "PT":
        final_cg.append(polymer_fragments["PT"])
        name = name[2:]
    if name[0:2] == "_PT":
        final_cg.append(polymer_fragments["PT"])
        name = name[2:]
    if name[0:4] == "CPDT":
        final_cg.append(polymer_fragments["CPDT"])
        name = name[4:]
    if name[0:10] == "CPDT_eneHD":
        final_cg.append(polymer_fragments["CPDT_eneHD"])
        name = name[10:0]
    for i in range(len(final_cg)):
        key = ("_"+letters[i]) # Formatting for GRiTS into dictionary with a letter as the key
        grits_string[key] = final_cg[i] # SMIILES string of the backbone segment as the value
    system = CG_System(
        filename,
        beads=grits_string,
        conversion_dict=element_dict
    )
    cg_gsd = "CG-" + new_gsd_name + ".gsd"
    system.save(cg_gsd)


In [9]:
backbones = cg("/home/jbieri/CME_lab/repos/pl-validation/gsd_files/PCPDTFBT_C4_BO_melt.gsd")

/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/grits/coarsegrain.py:153: UserWarning: c1ccncc1 not found in compound!
  warn(f"{smart_str} not found in compound!")
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/grits/coarsegrain.py:153: UserWarning: c1(F)ccc2nsnc2c1 not found in compound!
  warn(f"{smart_str} not found in compound!")
/home/jbieri/miniforge3/envs/p_l/lib/python3.11/site-packages/grits/coarsegrain.py:194: UserWarning: Some atoms have been left out of coarse-graining!
  warn("Some atoms have been left out of coarse-graining!")


In [5]:
print(backbones)

{'_A': 'c1ccncc1', '_B': 'C1C3=C(SC=C3)C2=C1C=CS2', '_C': 'c1(F)ccc2nsnc2c1'}


In [6]:
print(polymer_fragments)

{'P': 'c1ccncc1', 'CPDT': 'C1C3=C(SC=C3)C2=C1C=CS2', 'PT': 'n1ccc2nsnc2c1', 'IDT': 'C1C2=C(C3=CC4=C(C=C31)C5=C(C4)C=CS5)SC=C2', 'BT': 'c1ccc2nsnc2c1', 'DPP': 'N1C=C(C(=O)Nc2)c2C1(=O)', 'FBT': 'c1(F)ccc2nsnc2c1', 'T': 'c1ccsc1', 'CPDT_eneHD': 'C1(=C(CC(CCCCCC)CCCCCCCC)CC(CCCCCC)CCCCCCCC)C3=C(SC=C3)C2=C1C=CS2', 'TPD': 'C1(=O)C2=CSC=C2C(=O)N1(CCCCCCCC)', 'BDT': 's1c2c(OCC(CC)CCCC)c3ccsc3c(OCC(CC)CCCC)c2cc1'}


In [ ]:
original_traj = gsd.hoomd.open("/home/jbieri/CME_lab/repos/pl-validation/PCPDTFBT_C1_BOtest.gsd")
with gsd.hoomd.open("linear_traj.gsd","w") as new_traj:
    for frame in original_traj:
        new_traj.append(frame)